In [ ]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls
%mkdir data
!ls

gdrive	sample_data
data  gdrive  sample_data


In [ ]:
import sys

sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/machine learning/datasets')
sys.path.insert(1, '/content/gdrive/MyDrive/Colab Notebooks/machine learning/03 redes neuronales')

# from utils import displayData,

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

def featureNormalize(X_train, X_test):
    mu = np.mean(X_train, axis=0)
    sigma = np.std(X_train, axis=0)

    # Agrego este sigma en caso de desviación estándar cero para evitar divisiones por cero
    sigma[sigma == 0] = 1e-8

    # Aplico normalización
    X_train_normalized = (X_train - mu) / sigma
    X_test_normalized = (X_test - mu) / sigma

    return X_train_normalized, X_test_normalized, mu, sigma

# Cargo mi data set con 50000 filas y 56 columnas de clasificación binaria
data = np.loadtxt('/content/gdrive/MyDrive/Red Neuronal/preparado_1.2.txt', delimiter=';')

# Separo las características (X) y etiquetas (y)
X = data[:, :-1]
y = data[:, -1]

# Divido los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizo los datos de características
X_train_normalized, X_test_normalized, mu, sigma = featureNormalize(X_train, X_test)

# Imprimo las formas de los conjuntos de datos
print("Forma de X_train:", X_train_normalized.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de X_test:", X_test_normalized.shape)
print("Forma de y_test:", y_test.shape)



Forma de X_train: (46876, 55)
Forma de y_train: (46876,)
Forma de X_test: (11720, 55)
Forma de y_test: (11720,)


In [ ]:
import numpy as np

# Configurando parámetros necesarios para el siguiente paso
input_layer_size  = 55  # 55 características de entrada
hidden_layer_size = 20  # 20 unidades ocultas
num_labels = 2          # 2 etiquetas, binario (0 o 1)

#inicialiazo las capa de entrada y salida mas el vayas
Theta1 = np.random.rand(hidden_layer_size, input_layer_size + 1)  # +1 para el sesgo
Theta2 = np.random.rand(num_labels, hidden_layer_size + 1)         # +1 para el sesgo

# concateno
nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])

# Imprimir llos los valores de theta
print("Forma de Theta1:", Theta1.shape)
print("Forma de Theta2:", Theta2.shape)
print("Forma de nn_params:", nn_params.shape)


Forma de Theta1: (20, 56)
Forma de Theta2: (2, 21)
Forma de nn_params: (1162,)


In [ ]:
import numpy as np
#aplico las funciones de sigmoide y gradiente  y las aplico a la entrada , oculta , y salida
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoidGradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_=0.0):
    # los paraetros theta y theta
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1)))
    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):], (num_labels, (hidden_layer_size + 1)))

    m = y.size
    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    #
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
    z2 = a1.dot(Theta1.T)
    a2 = np.concatenate([np.ones((m, 1)), sigmoid(z2)], axis=1)
    z3 = a2.dot(Theta2.T)
    a3 = sigmoid(z3)


    y_matrix = np.eye(num_labels)[y]

    # funcion del costo
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(Theta1[:, 1:])) + np.sum(np.square(Theta2[:, 1:])))
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term


    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2[:,1:]) * sigmoidGradient(z2)

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)


    Theta1_grad = (1 / m) * Delta1
    Theta2_grad = (1 / m) * Delta2

    # se agrega el parametro de regulazacion
    Theta1_grad[:, 1:] += (lambda_ / m) * Theta1[:, 1:]
    Theta2_grad[:, 1:] += (lambda_ / m) * Theta2[:, 1:]

    # despues de apalnar
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad


In [ ]:
z = np.array([-1, -0.5, 0, 0.5, 1])
g = sigmoidGradient(z)
print('Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:\n  ')
print(g)

Gradiente sigmoide evaluada con [-1 -0.5 0 0.5 1]:
  
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [ ]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [ ]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [ ]:
def predict(Theta1, Theta2, X):
  #usar el 20% que es de test
    """
    Predict the label of an input given a trained neural network
    Outputs the predicted label of X given the trained weights of a neural
    network(Theta1, Theta2)
    """

    m = X.shape[0]
    num_labels = Theta2.shape[0]


    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [ ]:
# Predicciones para el conjunto de prueba
pred_test = predict(Theta1, Theta2, X_test_normalized)

# Calcular la precisión del conjunto de prueba
test_accuracy = np.mean(pred_test == y_test) * 100

# Imprimir la precisión del conjunto de prueba
print('Test Set Accuracy: %f' % test_accuracy)


Test Set Accuracy: 81.493174
